# Fallstudie Tablet GmbH

Finale Datei zur Abgabe am Ende des Projektes </br>


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import matplotlib.lines as mlines
import locale

## Datenaufbereitung

Der erste Schritt ist die Aufarbeitung der Datengrundlage, um eine Konsistente Nutzung dieser sicherzustellen. Hierzu werden folgende Punkte betrachtet:
<ul>
    <li>
        Spaltennamen konsistent
        <ul>
            <li>Englisch</li>
            <li>klein geschrieben</li>
            <li>keine Leerzeichen</li>
            <li>gleiches Prinzip bei Bezeichnung der Spalten (Snakecase)</li>
        </ul>
    </li>
    <li>Zum Wert passende Datentypen</li>
    <li>Hinzufügen weiterer Spalten</li>
    <li>Zusammenfügen von Daten verschiedener Tabellen</li>
</ul>

Für die Tabelle mit den Preisen müssen die Spaltennamen angepasst werden. Außerdem müssen die Werte der Jahresspalte bereinigt und der Datentyp angepasst werden.

In [3]:
df_price = (
    pd.read_excel("./data/Preise.xlsx") #./../../data/Preise.xlsx   - Pfad für Sandbox
        .rename(columns={"Jahr": "year",
                         "preis": "price"})
        .assign(year=lambda _data: pd.to_numeric(_data["year"].str.strip('"')))
        .replace([20022, "Tablet Basics"],[2022, "Tablet Basic"])
)

In der Downtime Tabelle müssen nur die Spaltennamen angepasst werden.

In [4]:
df_dt = (
    pd.read_excel("./data/Downtime.xlsx")
        .rename(columns={"Unnamed: 0": "year",
                         "Region A": "a",
                         "Region B": "b",
                         "Region C": "c",
                         "Region D": "d"})
)

In der Kundendaten Tabelle müssen auch nur die Spaltennamen angepasst werden. Eine Prüfung hat ergeben, dass es in dieser Tabelle keine Duplikate und keine Null Werte gibt.

In [5]:
df_crm = (
    pd.read_excel("./data/CRM_data.xlsx")
        .rename(columns={"id": "transaction_id",
                         "vorname": "firstname",
                         "nachname": "lastname"})
)

Bei den Tabellen mit den Transaktionsdaten können wir bei allen Tabellen gleich vorgehen. Daher macht es hier Sinn eine Funktion für die Aufarbeitung zu schreiben und diese für jede Datei aufzurufen. Hierbei müssen zunächst die Spaltennamen angepasst werden. Danach werden neue Spalten für das Jahr, den Monat und die Woche hinzugefügt, um bei der Analyse eine einfachere Einteilung in Zeitabschnitte zu bieten. Außerdem werden die Werte der Kosten auf zwei Nachkommastellen gerundet und es wird eine neue Spalte, welche immer die Kosten enthält, hinzugefügt. Danach werden noch Spalten wíe die Gesamtkosten der Transaktion, Umsatz und Gewinn hinzugefügt. Außerdem werden der geltende Preis für das Produkt und die Kundenbewertung der Transaktion der Tabelle hinzugefügt. </br>
Außerdem erstellen wir am Ende noch einen Dataframe in dem alle Regionen und Marktplätze zusammengefasst sind, um das ganze folgend zu untersuchen. Dazu werden den einzelnen Regionsdataframes noch Spalten für die Region und den Marktplatz hinzugefügt.

In [15]:
def read_regio_data(region, kind): 
    return (pd.read_excel(f"./data/Region{region}_{kind}.xlsx")
                .rename(columns={"Tablet Basic": "basic",
                                "Tablet Mini": "mini",
                                "Tablet Pro": "pro",
                                "kosten_mini": "cost_mini",
                                "kosten_basic": "cost_basic",
                                "kosten_pro": "cost_pro",
                                "PRODUKT": "product"})
                .assign(year=lambda _data: _data["date"].dt.year,
                        month=lambda _data: _data["date"].dt.month,
                        week=lambda _data: _data["date"].dt.isocalendar().week,
                        year_month=lambda _data: _data["date"].dt.year.astype(str) + "_" + _data["date"].dt.month.astype(str),
                        cost_basic=lambda _data: np.round(_data["cost_basic"], 2),
                        cost_mini=lambda _data: np.round(_data["cost_mini"], 2),
                        cost_pro=lambda _data: np.round(_data["cost_pro"], 2),
                        cost_per_unit=lambda _data: _data["cost_basic"].fillna(0) + _data["cost_mini"].fillna(0) + _data["cost_pro"].fillna(0),
                        cost=lambda _data: _data["cost_per_unit"] * _data["amount"]
                        )
                .merge(df_price, on=["year", "product"])
                .assign(revenue=lambda _data: _data["price"] * _data["amount"],
                        profit_per_unit=lambda _data: _data["price"] - _data["cost_per_unit"],
                        profit=lambda _data: _data["profit_per_unit"] * _data["amount"])
                .merge(df_crm[["customer_rating", "transaction_id"]], on="transaction_id")
                .assign(region = region,
                        marketplace = kind))

dataframes = [
    (df_regAo, 'A', 'online'),
    (df_regAp, 'A', 'phone'),
    (df_regAs, 'A', 'store'),
    (df_regBo, 'B', 'online'),
    (df_regBp, 'B', 'phone'),
    (df_regBs, 'B', 'store'),
    (df_regCo, 'C', 'online'),
    (df_regCp, 'C', 'phone'),
    (df_regCs, 'C', 'store'),
    (df_regDo, 'D', 'online'),
    (df_regDp, 'D', 'phone'),
    (df_regDs, 'D', 'store')
]

for df_reg, region, marketplace in dataframes:
    df_reg = read_regio_data(region,marketplace)



df_reg_main = pd.concat([df for df, _, _ in dataframes], ignore_index=True)